In [1]:
# -*- coding: utf-8
# Reinaldo Chaves (@paidatocandeira)
# Cria dataframes únicos para documentos de empenho e pagamento
# Faz cruzamentos para encontrar informaçõs dos fornecedores
# Filtra os gastos de combate da covid-19, que possuem 
# ['Código Ação'] == '21C0'

In [2]:
import pandas as pd
from pathlib import Path
import os
import csv

# Documentos de empenho, liquidação e pagamento

In [3]:
# Os zips de cada dia podem ser encontrados aqui: http://transparencia.gov.br/download-de-dados/despesas

In [4]:
# Em cada zip os arquivos que vamos usar são os que terminam por:

# Despesas_Empenho

# Despesas_Pagamento

# Despesas_Pagamento_EmpenhosImpactados

### Agrupa todos os arquivos de Despesas_Empenho em único dataframe

In [5]:
# Carrega as despesas empenhadas

In [6]:
print(os.getcwd())

/home/abraji/Documentos/Code/covid_19/repo/portal_transparencia


In [7]:
#test = pd.read_csv("zips_empenho_liquidacao_e_pagamento/Despesas_Empenho/20200602_Despesas_Empenho.csv", sep=';', encoding='latin_1', dtype = 'str')
#zips_empenho_liquidacao_e_pagamento/Despesas_Empenho/20200602_Despesas_Empenho.csv


In [8]:
#test.info()

In [9]:
def _reader_pasta(fname):
    return pd.read_csv(fname, sep=';', encoding='latin_1', dtype = 'str')

In [10]:
folder = Path("zips_empenho_liquidacao_e_pagamento_nova_versao/Despesas_Empenho")
despesas_empenho = pd.concat([
    _reader_pasta(csv)
    for csv in folder.glob("*.csv")
])

In [11]:
despesas_empenho.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1899047 entries, 0 to 9969
Data columns (total 63 columns):
 #   Column                                            Dtype 
---  ------                                            ----- 
 0   Id Empenho                                        object
 1   Código Empenho                                    object
 2   Código Empenho Resumido                           object
 3   Data Emissão                                      object
 4   Código Tipo Documento                             object
 5   Tipo Documento                                    object
 6   Tipo Empenho                                      object
 7   Espécie Empenho                                   object
 8   Código Órgão Superior                             object
 9   Órgão Superior                                    object
 10  Código Órgão                                      object
 11  Órgão                                             object
 12  Código Unidade Ge

### Separa pelo código do coronavírus

In [12]:
despesas_empenho_covid19 = despesas_empenho[(despesas_empenho['Código Ação'] == '21C0')]
despesas_empenho_covid19.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 90768 entries, 35 to 9953
Data columns (total 63 columns):
 #   Column                                            Non-Null Count  Dtype 
---  ------                                            --------------  ----- 
 0   Id Empenho                                        90768 non-null  object
 1   Código Empenho                                    90768 non-null  object
 2   Código Empenho Resumido                           90768 non-null  object
 3   Data Emissão                                      90768 non-null  object
 4   Código Tipo Documento                             90768 non-null  object
 5   Tipo Documento                                    90768 non-null  object
 6   Tipo Empenho                                      90768 non-null  object
 7   Espécie Empenho                                   90768 non-null  object
 8   Código Órgão Superior                             90768 non-null  object
 9   Órgão Superior              

### Agrupa todos os arquivos de Despesas_Pagamento_EmpenhosImpactados em único dataframe
### Carrega todos pagamentos de empenhos impactados 

In [13]:
# O problema é que o pagamento de um empenho pode ser numa data diferente
# Por isso é preciso baixar um grande intervalo de arquivos

In [14]:
folder = Path("zips_empenho_liquidacao_e_pagamento_nova_versao/Despesas_Pagamento_EmpenhosImpactados")
despesas_pagamento_empenhosimpactados = pd.concat([
    _reader_pasta(csv)
    for csv in folder.glob("*.csv")
])

In [15]:
despesas_pagamento_empenhosimpactados.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5517558 entries, 0 to 30160
Data columns (total 8 columns):
 #   Column                               Dtype 
---  ------                               ----- 
 0   Código Pagamento                     object
 1   Código Empenho                       object
 2   Código Natureza Despesa Completa     object
 3   Subitem                              object
 4   Valor Pago (R$)                      object
 5   Valor Restos a Pagar Inscritos (R$)  object
 6   Valor Restos a Pagar Cancelado (R$)  object
 7   Valor Restos a Pagar Pagos (R$)      object
dtypes: object(8)
memory usage: 378.9+ MB


### Cruza os dois dataframes pelo "Código Empenho"

In [16]:
despesas_pagamentos_empenho_covid19 = pd.merge(despesas_empenho_covid19, despesas_pagamento_empenhosimpactados, left_on='Código Empenho', right_on='Código Empenho')
despesas_pagamentos_empenho_covid19.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 83674 entries, 0 to 83673
Data columns (total 70 columns):
 #   Column                                            Non-Null Count  Dtype 
---  ------                                            --------------  ----- 
 0   Id Empenho                                        83674 non-null  object
 1   Código Empenho                                    83674 non-null  object
 2   Código Empenho Resumido                           83674 non-null  object
 3   Data Emissão                                      83674 non-null  object
 4   Código Tipo Documento                             83674 non-null  object
 5   Tipo Documento                                    83674 non-null  object
 6   Tipo Empenho                                      83674 non-null  object
 7   Espécie Empenho                                   83674 non-null  object
 8   Código Órgão Superior                             83674 non-null  object
 9   Órgão Superior              

In [17]:
# Pode cruzar com todas as despesas se quiser também

In [18]:
despesas_pagamentos_empenho_cruzado = pd.merge(despesas_empenho, despesas_pagamento_empenhosimpactados, left_on='Código Empenho', right_on='Código Empenho')
despesas_pagamentos_empenho_cruzado.shape

(4156223, 70)

In [19]:
#despesas_pagamentos_empenho_cruzado.info()

### Agora com esses pagamentos posso chegar no fornecedor e mais detalhes
### Tenho que abrir o arquivo de pagamentos finais

In [20]:
# Une em um dataframe só os pagamentos finais

In [21]:
folder = Path("zips_empenho_liquidacao_e_pagamento_nova_versao/Despesas_Pagamento")
despesas_pagamentos_finais = pd.concat([
    _reader_pasta(csv)
    for csv in folder.glob("*.csv")
])

In [22]:
despesas_pagamentos_finais.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5371335 entries, 0 to 22009
Data columns (total 34 columns):
 #   Column                                Dtype 
---  ------                                ----- 
 0   Código Pagamento                      object
 1   Código Pagamento Resumido             object
 2   Data Emissão                          object
 3   Código Tipo Documento                 object
 4   Tipo Documento                        object
 5   Tipo OB                               object
 6   Extraorçamentário                     object
 7   Código Órgão Superior                 object
 8   Órgão Superior                        object
 9   Código Órgão                          object
 10  Órgão                                 object
 11  Código Unidade Gestora                object
 12  Unidade Gestora                       object
 13  Código Gestão                         object
 14  Gestão                                object
 15  Código Favorecido                 

### Agora o cruzamento é pelo "Código Pagamento"

In [23]:
# Se quiser cruzar com todos os pagamentos
# pagamentos_feitos = pd.merge(despesas_pagamentos_empenho_cruzado, despesas_pagamentos_finais, left_on='Código Pagamento', right_on='Código Pagamento')

# Cruza com pagamentos da covid-19
pagamentos_feitos = pd.merge(despesas_pagamentos_empenho_covid19, despesas_pagamentos_finais, left_on='Código Pagamento', right_on='Código Pagamento')
pagamentos_feitos.shape

(83674, 103)

In [24]:
pagamentos_feitos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 83674 entries, 0 to 83673
Columns: 103 entries, Id Empenho to Valor Utilizado na Conversão_y
dtypes: object(103)
memory usage: 66.4+ MB


In [25]:
list(pagamentos_feitos.columns.values.tolist())

['Id Empenho',
 'Código Empenho',
 'Código Empenho Resumido',
 'Data Emissão_x',
 'Código Tipo Documento_x',
 'Tipo Documento_x',
 'Tipo Empenho',
 'Espécie Empenho',
 'Código Órgão Superior_x',
 'Órgão Superior_x',
 'Código Órgão_x',
 'Órgão_x',
 'Código Unidade Gestora_x',
 'Unidade Gestora_x',
 'Código Gestão_x',
 'Gestão_x',
 'Código Favorecido_x',
 'Favorecido_x',
 'Observação_x',
 'Código Esfera Orçamentária',
 'Esfera Orçamentária',
 'Código Tipo Crédito',
 'Tipo Crédito',
 'Código Grupo Fonte Recurso',
 'Grupo Fonte Recurso',
 'Código Fonte Recurso',
 'Fonte Recurso',
 'Código Unidade Orçamentária',
 'Unidade Orçamentária',
 'Código Função',
 'Função',
 'Código SubFunção',
 'SubFunção',
 'Código Programa',
 'Programa',
 'Código Ação',
 'Ação',
 'Linguagem Cidadã',
 'Código Subtítulo (Localizador)',
 'Subtítulo (Localizador)',
 'Código Plano Orçamentário_x',
 'Plano Orçamentário_x',
 'Código Programa Governo_x',
 'Nome Programa Governo_x',
 'Autor Emenda',
 'Código Categoria de 

### O campo "Código Favorecido" é o CNPJ ou CPF mascarado do fornecedor

In [26]:
pagamentos_feitos.to_csv('resultados/pagamentos_covid19/pagamentos_feitos_de_empenhos_covid19_ate_15_outubro_2020.csv', index=False)